- Extract wdate and reference

In [ ]:
import sqlite3
import pandas as pd
conn = sqlite3.connect('News.db')
head = pd.read_sql('SELECT * FROM head', conn)
conn.close()

In [ ]:
df = pd.read_excel('News_var.xlsx')
words = df[df.y == 1].iloc[:,1].drop_duplicates(keep='first') # 신어에 해당하는 것 가져오기
df = head

In [ ]:
import re
from string import punctuation

# Stopwords 처리
pattern1 = re.compile(r'[{}]'.format(re.escape(punctuation)))  # punctuation 제거
pattern2 = re.compile(r'[^가-힣 ]')  # 특수문자, 자음, 모음, 숫자, 영어 제거
pattern3 = re.compile(r'\s{2,}')  # white space 1개로 바꾸기.

df['head'] = df['head'].map(lambda x: pattern3.sub(' ',
                                                       pattern2.sub('',
                                                                    pattern1.sub('', x))))

In [ ]:
from collections import defaultdict
from tqdm import tqdm_notebook

# index length 만들기
length = list(range(len(df)))
length.pop(-1)
length = length[-1]

# 날짜와 reference 가져오기
sent = {w:[(df.iloc[_]['wdate'], df.iloc[_]['ref']) for _ in range(length) if w in df.iloc[_]['head']] for w in tqdm_notebook(words)}
# Excel에 저장
pd.DataFrame.from_dict(sent, orient='index').to_excel('wdate.xlsx')

- Clear Up Data

In [ ]:
# Data 원하는 형태로 가져오기
import ast
temp = pd.read_excel('wdate.xlsx', index_col='Unnamed: 0')
temp = temp.T.to_dict('list')
temp = {k:[ast.literal_eval(_) for _ in v if type(_) is str] for k,v in temp.items()}

In [ ]:
# date별, ref별로 dict 만들기
date = defaultdict(lambda:0)
ref = defaultdict(lambda:0)
for k, v in temp.items():
    dtemp = defaultdict(lambda:0)
    rtemp = defaultdict(lambda:0)
    for _ in v:
        if re.search(r'\d+[.]\d+',_[0]):
            dtemp[re.sub(r'[.]', r'-', re.search(r'\d+[.]\d+','2020.07').group())] += 1 # format 통일
        else:
            dtemp[re.search(r'\d+[-]\d+',_[0]).group()] += 1
        rtemp[_[1]] += 1
    date[k] = dtemp
    ref[k] = rtemp

In [ ]:
# nan 값이 있으면 sum이 무조건 nan으로 처리되는 것을 피하기 위해서 overriding
def sum(data):
    sum = 0
    for _ in data:
        if not math.isnan(_):
            sum += _
    return sum

In [ ]:
# ref별 비율 구하기
ref = pd.DataFrame.from_dict(ref, orient='index')
total_ref = defaultdict(lambda:0)
for _ in head['ref']:
    total_ref[_] += 1
    
for _ in ref:
    ref[_] = ref[_]/total_ref[_] # 각 빈도를 전체 수집 데이터로 나눈다.
    
for _ in ref.index:
    ref.loc[_] = ref.loc[_]/sum(ref.loc[_]) # 전체를 1로 맞추고, 각 비율을 구한다.

In [ ]:
pd.DataFrame.from_dict(date, orient='index').iloc[:,:13].to_excel('News_date.xlsx') # 1년치만 가져온다.
ref.to_excel('News_ref.xlsx')